In [1]:
# import function from file in another directory
import sys
sys.path.append('functions/')

from build_graph import build_graph
import networkx as nx
import numpy as np
import pandas as pd

## Build Graph

In [2]:
depth = 2
start_page = "Cumulative distribution function" 

graph, links_dict, categories_dict = build_graph(start_page, depth, display=False)

Graph found. Loading graph, links, and categories.
Number of nodes: 5480 , Number of edges: 469811


## Compute Jaccard Similarity Matrix

In [3]:
from neighbors import get_common_neighbors, get_total_neighbors, get_jaccard_coefficient

adjacency_matrix = nx.adjacency_matrix(graph).todense()
adjacency_matrix.astype(int)
common_neighbors_matrix = get_common_neighbors(adjacency_matrix)
total_neighbors_matrix = get_total_neighbors(adjacency_matrix, common_neighbors_matrix)
jaccard_similarity_matrix = get_jaccard_coefficient(common_neighbors_matrix, total_neighbors_matrix)

## Cluster the nodes with HDBSCAN

In [4]:
from dbscan import dbscan_from_similarity

# combine the adjacency matrix and the jaccard similarity matrix
clustering_matrix = jaccard_similarity_matrix + adjacency_matrix

# find the maximum value in the matrix
max_val = np.max(clustering_matrix)

# set the diagonal to the maximum value
np.fill_diagonal(clustering_matrix, max_val)

cluster_labels = dbscan_from_similarity(clustering_matrix)

Epsilon: 0.01, N Clusters: 98, Silhouette Score: -0.023
Epsilon: 0.041, N Clusters: 92, Silhouette Score: -0.014
Epsilon: 0.071, N Clusters: 97, Silhouette Score: -0.022
Epsilon: 0.102, N Clusters: 100, Silhouette Score: -0.04
Epsilon: 0.133, N Clusters: 104, Silhouette Score: -0.025
Epsilon: 0.163, N Clusters: 94, Silhouette Score: -0.022
Epsilon: 0.194, N Clusters: 79, Silhouette Score: 0.16
Epsilon: 0.225, N Clusters: 73, Silhouette Score: 0.142
Epsilon: 0.256, N Clusters: 72, Silhouette Score: 0.119
Epsilon: 0.286, N Clusters: 68, Silhouette Score: -0.016
Epsilon: 0.317, N Clusters: 61, Silhouette Score: -0.106

Subclustering the noise cluster

Subclustering the noise cluster

Subclustering the noise cluster

Subclustering the noise cluster
--------------------------------------------------
Final Silhouette Score: 0.12387587467884366
--------------------------------------------------


In [5]:
# get the number of clusters and the number of nodes in each cluster
n_clusters = len(set(cluster_labels))
cluster_sizes = [np.sum(cluster_labels == i) for i in range(-1, n_clusters)]
print(f"Number of clusters: {n_clusters}")
print(f'Cluster sizes: {cluster_sizes}')

max_size = max(cluster_sizes)
min_size = min(cluster_sizes)

print(f"Max cluster size: {max_size}, Min cluster size: {min_size}")

Number of clusters: 133
Cluster sizes: [238, 12, 5, 6, 5, 6, 5, 5, 7, 7, 8, 5, 5, 9, 6, 6, 6, 8, 8, 15, 5, 7, 6, 6, 21, 5, 6, 5, 5, 9, 63, 7, 5, 9, 7, 6, 9, 8, 5, 14, 8, 19, 8, 12, 11, 82, 7, 8, 5, 9, 5, 5, 14, 13, 33, 33, 5, 75, 452, 38, 126, 26, 88, 32, 27, 153, 127, 11, 28, 235, 529, 126, 36, 59, 17, 184, 9, 143, 182, 5, 5, 11, 11, 5, 8, 9, 5, 9, 6, 5, 798, 5, 21, 10, 5, 5, 12, 9, 5, 205, 6, 6, 12, 6, 5, 5, 11, 7, 5, 5, 6, 11, 8, 5, 7, 6, 494, 9, 9, 6, 11, 6, 5, 5, 7, 12, 7, 10, 15, 8, 7, 24, 20, 0]
Max cluster size: 798, Min cluster size: 0


In [6]:
# find the cluster of the start page
start_page_cluster = cluster_labels[0]

# find the nodes in the same cluster as the start page
start_page_cluster_nodes = [node for node, cluster in zip(graph.nodes, cluster_labels) if cluster == start_page_cluster]

# find the dimension of the start page cluster
start_page_cluster_dim = len(start_page_cluster_nodes)

print(f"{start_page_cluster_dim} nodes in cluster {start_page_cluster}, with the start page:")
for node in start_page_cluster_nodes:
    print(node)

798 nodes in cluster 89, with the start page:
Cumulative distribution function
Characteristic function (probability theory)
Complex random variable
Continuous function
Continuous variable
Càdlàg
Derivative
Expected value
Interval (mathematics)
Inverse transform sampling
L1-norm
Lebesgue integral
Moment-generating function
Multivariate random variable
Normal distribution
Probability
Probability density function
Probability mass function
Probability theory
Random number generation
Random variable
Random vector
Riemann–Stieltjes integral
Right-continuous
Statistics
Bounded variation
Calculus
Calculus of variations
Compact space
Convergence of random variables
Differentiable function
Distributional derivative
Equivalence class
Function (mathematics)
Functional analysis
Generalized function
Haar measure
Hilbert space
Hölder condition
Integral
Lebesgue integration
Lebesgue measure
Lipschitz continuity
Lipschitz continuous
Lp space
Measure (mathematics)
Measure theory
Metric space
Probability

## Find missing link candidates

In [7]:
# set the diagonal of the adjacency matrix to 0
np.fill_diagonal(adjacency_matrix, 0)
boolean_adjacency_matrix = adjacency_matrix > 0
masked_similarity_matrix = jaccard_similarity_matrix[boolean_adjacency_matrix]

# find weak quantile threshold
weak_quantile_threshold = 0.75
weak_threshold = np.quantile(masked_similarity_matrix, weak_quantile_threshold)

# find strong quantile threshold
strong_quantile_threshold = 0.95
strong_threshold = np.quantile(masked_similarity_matrix, strong_quantile_threshold)

# print the thresholds
print(f'Similarity thresholds: ')
print(f'Weak quantile threshold: {weak_threshold}')
print(f'Strong quantile threshold: {strong_threshold}')

Similarity thresholds: 
Weak quantile threshold: 0.6183860019850799
Strong quantile threshold: 0.9274193548387096


In [8]:
from missing_links import find_missing_links_multi_thread

missing_link_candidates, missing_link_candidates_matrix = find_missing_links_multi_thread(graph, jaccard_similarity_matrix, cluster_labels, weak_threshold, strong_threshold)

print(f"Number of missing link candidates: {len(missing_link_candidates)}")

Number of missing link candidates: 65628


## Build dataset to train XGBoost

In [9]:
from build_dataset import build_dataset_multi_thread

train_df, filtered_categories_dict = build_dataset_multi_thread(adjacency_matrix, jaccard_similarity_matrix, missing_link_candidates_matrix, 
                                                    common_neighbors_matrix, total_neighbors_matrix, 
                                                    graph, cluster_labels, categories_dict, df_type='train')

missing_link_df, filtered_categories_dict = build_dataset_multi_thread(adjacency_matrix, jaccard_similarity_matrix, missing_link_candidates_matrix,
                                                            common_neighbors_matrix, total_neighbors_matrix,graph, cluster_labels, categories_dict,
                                                            df_type='missing links', filtered_categories_dict=filtered_categories_dict)

In [10]:
from sklearn.model_selection import train_test_split

# split the dataset into train and test
X = train_df.drop(columns=['node_1', 'node_2', 'link'])
y = train_df['link']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Train XGBoost

In [11]:
from xgboost import XGBClassifier
from tune_model import tune
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score


# train the model
model = XGBClassifier()

# calculate scale_pos_weight
scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)

# set parameters for grid search
space = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'scale_pos_weight': [scale_pos_weight],
    'objective': ['binary:logistic'],
    'alpha': [0, 0.1]
}

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

best_params, best_model = tune(X, y, space, scoring, 
                            model, modeltype='clf', search_type='grid', n_iter_random=100,
                            n_splits=2, n_repeats=1, random_state=1,
                            verbose=True, display_plots=0, refit='roc_auc')


Fitting 2 folds for each of 12 candidates, totalling 24 fits

Best Score (roc_auc): 0.984842903323654
accuracy: 0.9441893005270152
precision: 0.3525468083639737
recall: 0.927810386627536
f1: 0.5109379444090021

Best Hyperparameters:
alpha: 0.1
max_depth: 3
n_estimators: 100
objective: binary:logistic
scale_pos_weight: 31.024257153338226

Optimal Threshold: 0.4919257


In [12]:
# test the model
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")
print(f"ROC AUC: {roc_auc}")
print(f"Confusion matrix: ")
print(confusion)

Accuracy: 0.9430889148004237
Precision: 0.35129772204143184
Recall: 0.9278825995807128
F1 score: 0.509643617940008
ROC AUC: 0.9357360742486319
Confusion matrix: 
[[273423  16346]
 [   688   8852]]


## Find Missing Links

In [13]:
# save the columns 'node1', 'node2' into arrays
node1 = missing_link_df['node_1'].values
node2 = missing_link_df['node_2'].values

# remove the columns 'node1', 'node2' from the dataframe
missing_link_to_preidct_df = missing_link_df.drop(columns=['node_1', 'node_2'])

# predict link probabilities for the missing link candidates
link_probabilities = best_model.predict_proba(missing_link_to_preidct_df)[:, 1]

# create a new dataframe with the columns 'node1', 'node2', 'link_probability', 'similarity' and clusters of the nodes
missing_link_predictions_df = pd.DataFrame({
    'node_1': node1, 
    'node_2': node2, 
    'link_probability': link_probabilities, 
    'similarity': missing_link_df['similarity'].values, 
    'cluster_node_1': missing_link_df['cluster_node_1'].values, 
    'cluster_node_2': missing_link_df['cluster_node_2'].values})

# sort the dataframe by link_probability
missing_link_predictions_df = missing_link_predictions_df.sort_values(by='link_probability', ascending=False)

# keep only the values above 0.5 probability
missing_link_predictions_df = missing_link_predictions_df[missing_link_predictions_df['link_probability'] > 0.5]

# set pd printing limits to display all the rows
pd.set_option('display.max_rows', None)
columns_to_display = ['node_1', 'node_2', 'link_probability']
print("Missing link predictions: ")
missing_link_predictions_df[columns_to_display].head(100)

Missing link predictions: 


,node_1,node_2,link_probability
64257,Statistical software,List of open source statistical packages,0.999947
42310,Information engineering,Information engineering (field),0.999932
56612,Case–control study,Case-control study,0.999930
46569,Mechanical engineer,Mechanical engineering,0.999916
57478,Health Canada,List of national public health agencies,0.999910
57477,Health Canada,Health departments in the United States,0.999910
57480,Health Canada,National public health institutes,0.999910
57487,Health departments in the United States,Public Health Agency of Canada,0.999906
52508,Robotic,Robotics,0.999894
46553,Materials engineering,Materials science,0.999892


In [14]:
# find missing links where either node1 or node2 are in the same cluster as the start page
start_page_cluster = cluster_labels[0]
node1_cluster_mask = missing_link_predictions_df['cluster_node_1'].values == start_page_cluster
node2_cluster_mask = missing_link_predictions_df['cluster_node_2'].values == start_page_cluster
start_page_cluster_mask = node1_cluster_mask | node2_cluster_mask

start_page_cluster_missing_links = missing_link_predictions_df[start_page_cluster_mask]

print(f'Missing links where either node1 or node2 are in the same cluster as the start page:')
start_page_cluster_missing_links[columns_to_display].head(100)


Missing links where either node1 or node2 are in the same cluster as the start page:


,node_1,node_2,link_probability
58841,Point-set topology,General topology,0.998603
4114,Closed interval,Open interval,0.998512
2742,Continuous dual,Continuous dual space,0.998410
4240,Lebesgue integrability condition,Riemann-integrable,0.998358
4238,Lebesgue integrability condition,Riemann integrable,0.998358
4304,Open interval,Semi-open interval,0.998300
4327,Supremum,Infimum,0.998185
56357,Riemann integrable,Riemann-integrable,0.998149
4115,Closed interval,Semi-open interval,0.998051
1083,Riemann integration,Riemann-integrable,0.997971


In [15]:
# find missing links that include the start page if node1 or node2 is the start page
missing_links_start_page_df = missing_link_predictions_df[(missing_link_predictions_df['node_1'] == start_page) | (missing_link_predictions_df['node_2'] == start_page)]

print(f'Missing links that include the start page:')
missing_links_start_page_df[columns_to_display].head(100)

Missing links that include the start page:


,node_1,node_2,link_probability


In [16]:
# find missing links between nodes in different clusters
different_cluster_mask = missing_link_predictions_df['cluster_node_1'].values != missing_link_predictions_df['cluster_node_2'].values

different_cluster_missing_links_df = missing_link_predictions_df[different_cluster_mask]

print(f'Missing links between nodes in different clusters:')
different_cluster_missing_links_df[columns_to_display].head(100)

Missing links between nodes in different clusters:


,node_1,node_2,link_probability
4240,Lebesgue integrability condition,Riemann-integrable,0.998358
4238,Lebesgue integrability condition,Riemann integrable,0.998358
4324,Semi-open interval,Subinterval,0.989826
61906,Analytical theory of probabilities,Pierre Simon de Laplace,0.955510
63457,Misuse of p-values,P-value fallacy,0.948002
53969,Sports equipment,Spotting scope,0.817114
53970,Sports equipment,Trapping,0.810717
51931,Ready-mix concrete,Seed company,0.738985
40756,Homesteading,Seed company,0.738985
45958,Maid service,Seed company,0.704100
